In [6]:
import redis
import faker as fk
import pandas
import random
fake = fk.Faker()

# main client
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

In [7]:
def make_rand_computer():
    os = ['windows', 'macOS', 'linux']
    return {"ip": fake.ipv4(), "name": fake.user_name(), "os": random.choice(os)}

def make_rand_mac():
    return fake.mac_address()

def unravel_dict(dictionary):
    return " ".join([f"{key} {value}"for key,value in dictionary.items()])

def add_computer(computer):
    mac = make_rand_mac()
    create = f"HSET mac:{mac} {unravel_dict(computer)}"
    by_ip = f"HSET index.mac.by.ip {computer['ip']} mac {mac}"
    connections = f""
    macs = list()
    for key in r.scan_iter("mac:*"):
        macs.append(key)
    
    for _ in range(4):
        if macs == []:
            break
        connected = random.choice(macs)
        connections += f"""
                        SADD connections:{mac} {connected}
                        SADD connections:{connected} {mac}
                        """
        
    return create + by_ip + connections

print(add_computer(make_rand_computer()))

ConnectionError: Error 10061 connecting to localhost:6379. Nie można nawiązać połączenia, ponieważ komputer docelowy aktywnie go odmawia.

'ip 36.211.164.226 name dking os macOS'